# Mini-Projet Intelligence Artificielle : Prédiction du Revenu Annuel d'un Marocain

**Membres du Groupe :**
*   Saad Barhrouj
*   Nassim El Kaddaoui
*   Youness Kihel

**Année :** 2ème année Cycle d’Ingénieurs – GI
**Module :** Intelligence Artificielle
**Encadrant :** Y. EL YOUNOUSSI
**Année Universitaire :** 2024-2025
**Date de présentation :** 17 mai 2025 

---

*Ce notebook présente notre travail pour le mini-projet d'Intelligence Artificielle. Il détaille chaque étape de la construction d'un modèle de prédiction du revenu annuel des Marocains, depuis la génération des données jusqu'au déploiement.*

L'objectif général de ce mini-projet est de construire un pipeline complet de Machine Learning en Python pour prédire le revenu annuel des Marocains à partir de données simulées réalistes. Le projet couvre toutes les étapes de développement d’un modèle de Machine Learning : compréhension des données, préparation des données, modélisation, évaluation et déploiement.

# 0. Génération du Dataset Synthétique

Avant de pouvoir analyser et modéliser, nous avons dû générer un dataset synthétique simulant les revenus des Marocains. Cette étape a été réalisée via un script Python séparé (`generate_dataset.py`). Ci-dessous, nous résumons la méthodologie et les principales logiques implémentées dans ce script.

**Objectif du script `generate_dataset.py` :**
Créer un fichier CSV (`dataset_revenu_marocains.csv`) contenant environ 40 000 enregistrements, en respectant des contraintes déduites des statistiques du HCP et des spécifications du projet.

**Principales caractéristiques du dataset généré :**
*   **Variable Cible :** `Revenu_Annuel`.
*   **Caractéristiques Explicatives :** Incluent `Age`, `Categorie_age`, `Milieu` (Urbain/Rural), `Sexe`, `Niveau_education`, `Annees_experience`, `Etat_matrimonial`, `Possession_biens` (Immobilier, Véhicule, Terrain), `CSP`, ainsi que des caractéristiques supplémentaires (`Region_geographique`, `Secteur_emploi`, `Revenu_secondaire`).
*   **Imperfections Intégrées :** Le dataset inclut des valeurs manquantes, des valeurs aberrantes, des colonnes redondantes et des colonnes non pertinentes, conformément aux exigences du projet.

## 0.1 Logique de Génération et Statistiques Clés (Résumé)

Le script `generate_dataset.py` utilise la graine aléatoire `RANDOM_SEED = 42` pour assurer la reproductibilité des résultats et est configuré pour générer `N_RECORDS = 40000` enregistrements.

**Logique de Génération (Principes Généraux) :**

1.  **Caractéristiques de base :** Des attributs comme `Age`, `Milieu`, `Sexe`, et `Niveau_education` sont générés en premier, souvent en utilisant des distributions de probabilités prédéfinies (par exemple, une proportion cible de `P_URBAIN = 0.60` pour le milieu).
2.  **Caractéristiques dépendantes :** D'autres attributs tels que `Annees_experience` (dérivée de l'âge et du niveau d'éducation), `Etat_matrimonial` (influencé par l'âge), `CSP` (déterminée par l'éducation, l'expérience et l'âge), et la `Possession_biens` (liée à la CSP et au milieu) sont générés de manière à simuler des interdépendances réalistes.
3.  **Revenu Annuel (Variable Cible) :** La génération du revenu est l'étape la plus sophistiquée.
    *   Elle part d'une **distribution log-normale de base**, différenciée pour les milieux urbain et rural.
    *   Ce revenu de base est ensuite ajusté en fonction de multiples facteurs (`Milieu`, `Niveau_education`, `Annees_experience`, `CSP`, `Sexe`, `Region_geographique`, `Secteur_emploi`, `Revenu_secondaire`) via des bonus additifs et des facteurs multiplicatifs.
    *   Des étapes de **calibration** sont appliquées pour aligner les moyennes de revenus (urbain/rural) avec les cibles HCP.
    *   Une **transformation de puissance** est utilisée pour ajuster la distribution afin de respecter les proportions cibles de revenus inférieurs aux moyennes (globale, urbaine, rurale).
    *   Un plancher et un plafond sont appliqués pour maintenir les revenus dans une fourchette plausible, avec un traitement spécifiques pour les inactifs.
4.  **Introduction des Imperfections :**
    *   **Valeurs manquantes (`NaN`) :** Introduites avec une faible probabilité dans certaines colonnes (ex: `Etat_matrimonial`, `Secteur_emploi`, `Annees_experience`, `Possession_biens`).
    *   **Valeurs aberrantes :** Ajoutées de manière contextuelle sur `Age` (ex: -5, 150), `Annees_experience` (ex: expérience supérieure à l'âge), `Possession_biens` (ex: inactif avec un patrimoine important, ou agriculteur sans terrain), et `Revenu_Annuel` (revenus extrêmes ou incohérents avec la CSP, comme des revenus très bas pour les inactifs ou très élevés pour les cadres).
    *   **Colonnes redondantes/non pertinentes :** `Revenu_Mensuel` (calculé directement à partir de `Revenu_Annuel`), `Adresse_Email` (identifiant unique formaté) et `CIN` (identifiant unique simulé).
5.  **Colonne dérivée :** `Categorie_age` ('Jeune', 'Adulte', 'Senior', 'Âgé') est créée à partir de la variable `Age` pour faciliter certaines analyses.

**Statistiques Clés du Dataset `dataset_revenu_marocains.csv` (issues de l'exécution de `generate_dataset.py`) :**

*   **Nombre d'enregistrements :** 40,000
*   **Revenu Annuel Moyen (global) :** 21,506 DH *(Cible HCP: 21,949 DH)*
*   **Écart-type du Revenu Annuel :** 33,809 DH
*   **Revenu Annuel (Min | 25% | Médiane | 75% | Max) :** 200 DH | 2,211 DH | 10,501 DH | 26,107 DH | 700,000 DH
*   **Moyenne Revenu Urbain :** 27,157 DH *(Cible HCP: 26,988 DH)*
*   **Moyenne Revenu Rural :** 13,084 DH *(Cible HCP: 12,862 DH)*
*   **Proportion revenus < moyenne globale :** 70.5% *(Cible HCP: 71.8%)*
*   **Proportion revenus < moyenne urbaine :** 66.0% *(Cible HCP: 65.9%)*
*   **Proportion revenus < moyenne rurale :** 84.7% *(Cible HCP: 85.4%)*
*   **Classement des revenus moyens par CSP (décroissant, DH) :**
    *   Cadres supérieurs : 107,018
    *   Professions intermédiaires : 32,222
    *   Employés : 16,967
    *   Ouvriers : 7,998
    *   Agriculteurs : 6,004
    *   Inactifs : 2,692
*   **Classement des revenus moyens par Niveau d'éducation (décroissant, DH) :**
    *   Supérieur : 60,561
    *   Secondaire : 19,049
    *   Fondamental : 7,174
    *   Sans niveau : 4,833

Pour une compréhension exhaustive et détaillée de la logique de génération, le script `generate_dataset.py` reste la référence principale. Le dataset ainsi créé est celui que nous allons charger et analyser dans les sections suivantes de ce notebook.

# 1. Compréhension des Données

Après avoir résumé le processus de génération du dataset, nous allons maintenant charger le fichier `dataset_revenu_marocains.csv` et l'explorer en détail. L'objectif est de bien comprendre sa structure, son contenu, et d'identifier les caractéristiques notables avant de passer à la préparation des données pour la modélisation.

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_SEED = 42 
np.random.seed(RANDOM_SEED)

# Options d'affichage 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 100)    
pd.set_option('display.width', 1000)    

# Pour des graphiques plus esthétiques 
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("pastel")

# Ignorer les avertissements pour un notebook plus propre
import warnings
warnings.filterwarnings('ignore')

print("Librairies pour l'analyse de données importées.")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Seaborn version: {sns.__version__}")

Librairies pour l'analyse de données importées.
Pandas version: 2.2.3
Numpy version: 2.1.3
Seaborn version: 0.13.2


## 1.1 Chargement du Dataset
Nous chargeons le fichier `dataset_revenu_marocains.csv` qui a été généré par notre script `generate_dataset.py`. Ce fichier contient les données simulées sur lesquelles nous allons travailler.

In [58]:
FILENAME = "dataset_revenu_marocains.csv"
df = None 

try:
    df = pd.read_csv(FILENAME)
    print(f"Le fichier '{FILENAME}' a été chargé avec succès dans le DataFrame 'df'.")
    print(f"Dimensions du DataFrame : {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print(f"ERREUR CRITIQUE : Le fichier '{FILENAME}' n'a pas été trouvé.")
    print("Veuillez vérifier que le script de génération a bien été exécuté et que le fichier CSV est au bon emplacement (généralement le même dossier que ce notebook).")
    print("Les étapes suivantes ne pourront pas être exécutées sans le chargement des données.")
except Exception as e:
    print(f"Une erreur est survenue lors du chargement du fichier '{FILENAME}' :")
    print(e)
    print("Les étapes suivantes ne pourront pas être exécutées sans le chargement des données.")

if df is not None:
    print("\nLe DataFrame 'df' est prêt pour l'analyse.")
else:
    print("\nATTENTION : Le DataFrame 'df' n'a pas été chargé. Veuillez corriger le problème avant de continuer.")

Le fichier 'dataset_revenu_marocains.csv' a été chargé avec succès dans le DataFrame 'df'.
Dimensions du DataFrame : 40000 lignes et 18 colonnes.

Le DataFrame 'df' est prêt pour l'analyse.


## 1.2 Affichage des Données (Aperçu)
Pour avoir une première idée concrète du contenu et de la structure des données, affichons les 10 premières et les 10 dernières lignes du DataFrame. Cela nous permet de vérifier rapidement les noms des colonnes et le format des données.

In [59]:
if df is not None:
    print("Aperçu des 10 premières lignes du DataFrame :")
    display(df.head(10))

    print("\nAperçu des 10 dernières lignes du DataFrame :")
    display(df.tail(10))
else:
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible d'afficher les données.")

Aperçu des 10 premières lignes du DataFrame :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Annuel,Revenu_Mensuel,Adresse_Email,CIN
0,56,Senior,Homme,Rural,Centre,Marié,Fondamental,5.0,Agriculteurs,Privé,Non,Non,Oui,Non,300.000000,25.00,user748aa2@example.com,YF632342
1,46,Senior,Femme,Urbain,Centre,Célibataire,Supérieur,14.0,Cadres supérieurs,Public,Oui,Oui,Non,Non,78506.710725,6542.23,user861b7d@example.com,DU412942
2,32,Adulte,Homme,Rural,Sud,Marié,Fondamental,8.0,Ouvriers,Informel,Non,Oui,Non,Non,395.248250,32.94,user0e21f6@example.com,UQ738551
3,60,Âgé,Homme,Rural,Sud,Marié,Supérieur,17.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui,105781.533607,8815.13,user91e502@example.com,GE492077
4,25,Adulte,Femme,Urbain,Centre,Marié,Secondaire,3.0,Employés,Privé,Non,Non,Non,Non,22016.268128,1834.69,userdb161f@example.com,YF665579
5,38,Adulte,Homme,Rural,Est,Divorcé,Secondaire,5.0,Employés,Privé,Oui,Non,Non,Non,1995.320311,166.28,userf12e9f@example.com,YQ100599
6,56,Senior,Homme,Rural,Ouest,Marié,Fondamental,33.0,Ouvriers,Privé,Non,Non,Non,Non,1176.015258,98.00,user782a30@example.com,TK612340
7,36,Adulte,Femme,Urbain,Ouest,Marié,Secondaire,7.0,Employés,Privé,Oui,Oui,Non,Non,22195.154959,1849.60,user0c5223@example.com,AD480612
8,40,Adulte,Homme,Urbain,Centre,Marié,Supérieur,13.0,Cadres supérieurs,Privé,Oui,Non,Non,Non,92564.488887,7713.71,user241e19@example.com,ZJ351083
9,28,Adulte,Femme,Urbain,Nord,Marié,Secondaire,3.0,Ouvriers,Privé,Non,Non,Non,Non,17172.720964,1431.06,user6bfb9f@example.com,BH694916



Aperçu des 10 dernières lignes du DataFrame :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Annuel,Revenu_Mensuel,Adresse_Email,CIN
39990,26,Adulte,Homme,Urbain,Nord,Veuf,Secondaire,5.0,Employés,Public,Non,Non,Non,Oui,22982.650695,1915.22,userf32442@example.com,YJ411569
39991,40,Adulte,Homme,Rural,Nord,Célibataire,Secondaire,1.0,Ouvriers,Privé,Non,Non,Non,Non,1123.483370,93.62,user8e8e2c@example.com,PN139847
39992,24,Jeune,Femme,Urbain,Sud,Célibataire,Fondamental,1.0,Agriculteurs,Informel,Non,Non,Oui,Oui,11724.909786,977.08,usereef9f8@example.com,UP616808
39993,48,Senior,Femme,Urbain,Centre,Marié,Secondaire,25.0,Professions intermédiaires,Public,Non,Non,Oui,Non,38373.940680,3197.83,userbe625c@example.com,TQ978793
39994,38,Adulte,Femme,Urbain,Nord,Célibataire,Supérieur,13.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui,72210.412302,6017.53,userd32e41@example.com,SQ690182
39995,18,Jeune,Femme,Urbain,Centre,Marié,Sans niveau,0.0,Inactifs,NaN,Non,Non,Non,Non,2652.218311,221.02,useref70b8@example.com,MV264884
39996,63,Âgé,Homme,Urbain,Ouest,Divorcé,Supérieur,30.0,Cadres supérieurs,Public,Non,Oui,Non,Oui,146859.531881,12238.29,user5d6c5c@example.com,QN622528
39997,19,Jeune,Femme,Rural,Sud,Célibataire,Fondamental,0.0,Inactifs,NaN,Non,Non,Non,Non,300.000000,25.00,userea297d@example.com,JG342827
39998,57,Senior,Homme,Urbain,Nord,Marié,Secondaire,21.0,Professions intermédiaires,Privé,Non,Non,Non,Oui,47636.743845,3969.73,useref6bca@example.com,GW748819
39999,28,Adulte,Homme,Urbain,Nord,Marié,Sans niveau,1.0,Inactifs,NaN,Oui,Non,Non,Oui,3926.534904,327.21,user87b9ed@example.com,UR878598


## 1.3 Description Générale des Données (Types et Valeurs Manquantes)
Pour obtenir une vue synthétique des types de données de chaque colonne et du nombre de valeurs non nulles (ce qui nous indique les valeurs manquantes), nous utilisons la méthode `info()` du DataFrame.

In [ ]:
if df is not None:
    print("Informations générales sur le DataFrame (types, nombre de valeurs non nulles par colonne) :")
    df.info()
else:
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible d'afficher les informations.")

Informations générales sur le DataFrame (types, nombre de valeurs non nulles par colonne) :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    40000 non-null  int64  
 1   Categorie_age          39960 non-null  object 
 2   Sexe                   40000 non-null  object 
 3   Milieu                 40000 non-null  object 
 4   Region_geographique    40000 non-null  object 
 5   Etat_matrimonial       39965 non-null  object 
 6   Niveau_education       40000 non-null  object 
 7   Annees_experience      39960 non-null  float64
 8   CSP                    40000 non-null  object 
 9   Secteur_emploi         37390 non-null  object 
 10  Propriete_immobiliere  39958 non-null  object 
 11  Vehicule_motorise      39965 non-null  object 
 12  Terrain_agricole       39969 non-null  object 
 13  Revenu_seconda

## 1.4 Statistiques Descriptives
Nous allons maintenant calculer les statistiques descriptives (moyenne, écart-type, min, max, quartiles, etc.) pour les variables numériques, et les statistiques de fréquence (nombre d'uniques, valeur la plus fréquente) pour les variables catégorielles.

In [60]:
if df is not None:
    print("Statistiques descriptives pour les colonnes numériques :")
    numeric_stats = df.select_dtypes(include=np.number).describe().T
    display(numeric_stats)

    print("\nStatistiques descriptives pour les colonnes catégorielles (type 'object' et 'category') :")
    categorical_stats = df.select_dtypes(include=['object', 'category']).describe().T
    display(categorical_stats)

    print("\nNombre de valeurs uniques par colonne (trié par ordre décroissant) :")
    unique_counts = df.nunique().sort_values(ascending=False).to_frame(name='Nombre de Valeurs Uniques')
    display(unique_counts)
else:
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible de calculer les statistiques.")

Statistiques descriptives pour les colonnes numériques :


,count,mean,std,min,25%,50%,75%,max
Age,40000.0,40.512625,13.509623,-5.00,29.000000,40.000000,52.000000,150.00
Annees_experience,39960.0,10.401051,9.890290,0.00,2.000000,8.000000,16.000000,45.00
Revenu_Annuel,40000.0,21505.790212,33809.163803,200.00,2211.446624,10500.621766,26107.364841,700000.00
Revenu_Mensuel,40000.0,1792.149185,2817.430317,16.67,184.287500,875.050000,2175.612500,58333.33



Statistiques descriptives pour les colonnes catégorielles (type 'object' et 'category') :


,count,unique,top,freq
Categorie_age,39960,4,Adulte,17615
Sexe,40000,2,Homme,20817
Milieu,40000,2,Urbain,23937
Region_geographique,40000,5,Centre,11212
Etat_matrimonial,39965,4,Marié,21297
Niveau_education,40000,4,Secondaire,13974
CSP,40000,6,Ouvriers,14975
Secteur_emploi,37390,3,Privé,17624
Propriete_immobiliere,39958,2,Non,29202
Vehicule_motorise,39965,2,Non,28477



Nombre de valeurs uniques par colonne (trié par ordre décroissant) :


,Nombre de Valeurs Uniques
CIN,39999
Adresse_Email,39956
Revenu_Annuel,34300
Revenu_Mensuel,32396
Age,48
Annees_experience,46
CSP,6
Region_geographique,5
Categorie_age,4
Etat_matrimonial,4


## 1.5 Exploration Visuelle Approfondie (EDA avec ydata-profiling)
Pour obtenir une compréhension encore plus profonde et visuelle des données, notamment les distributions de chaque variable, les corrélations, les interactions et les valeurs manquantes, nous allons utiliser la librairie `ydata-profiling` (anciennement Pandas Profiling). Elle génère un rapport HTML interactif qui fournit une analyse détaillée de chaque variable du dataset.


In [ ]:
if 'df' in locals() and df is not None: 
    try:
        from ydata_profiling import ProfileReport
        
        print("Génération du rapport avec ydata-profiling... Cela peut prendre quelques instants.")
        
        profile = ProfileReport(df, 
                                title="Rapport d'Exploration Détaillée - Revenu des Marocains",
                                explorative=True) 
        
        profile_filename = "Pandas_Profiling_Report_Revenu_Marocains.html"
        profile.to_file(profile_filename)
        
        print(f"\nRapport ydata-profiling généré et sauvegardé sous : '{profile_filename}'")
        print(f"Veuillez ouvrir le fichier '{profile_filename}' manuellement depuis votre explorateur de fichiers pour le consulter.")

    except ImportError:
        print("ERREUR d'importation: La librairie ydata-profiling n'est pas installée.")
        print("Veuillez l'installer (par exemple, avec : pip install ydata-profiling) dans l'environnement Python utilisé par VS Code.")
    except Exception as e:
        print(f"Une erreur générale est survenue lors de la génération du rapport ydata-profiling : {e}")
else:
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé ou n'est pas défini. Veuillez exécuter les cellules précédentes pour charger 'df'.")

Génération du rapport avec ydata-profiling... Cela peut prendre quelques instants.


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:01<00:00, 11.50it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]


Rapport ydata-profiling généré et sauvegardé sous : 'Pandas_Profiling_Report_Revenu_Marocains.html'
Veuillez ouvrir le fichier 'Pandas_Profiling_Report_Revenu_Marocains.html' manuellement depuis votre explorateur de fichiers pour le consulter.


## 1.6 Bilan de la Compréhension des Données

Après avoir examiné les statistiques descriptives (`df.info()`, `df.describe()`) et le rapport d'exploration détaillé généré par `ydata-profiling` (version v4.16.1), nous pouvons synthétiser nos principales observations concernant le dataset :

*   **Structure Générale :**
    *   Le dataset contient **40 000 observations** et **18 variables**.
    *   Il a été confirmé par le rapport qu'il n'y a **aucune ligne dupliquée** (`Duplicate rows: 0`).
    *   Le nombre total de cellules manquantes est de **2871**, ce qui représente **0.4%** de l'ensemble des données (`Missing cells: 2871`, `Missing cells (%): 0.4%`).

*   **Variable Cible (`Revenu_Annuel`) :**
    *   Elle présente une distribution fortement asymétrique à droite (Skewness = **4.53**), avec une Kurtosis élevée (**39.18**), indiquant des queues épaisses (plus d'outliers). La moyenne (Mean) est d'environ **21 506 DH**, tandis que la médiane est significativement plus basse, à environ **10 501 DH**. Cette asymétrie est typique des données de revenu.
    *   Les valeurs s'étendent de **200 DH** (Minimum) à **700 000 DH** (Maximum).
    *   Aucune valeur manquante n'est présente pour cette variable cible (`Missing: 0`).

*   **Variables Explicatives Numériques :**
    *   `Age` : Les valeurs varient de **-5** (Minimum) à **150** ans (Maximum), confirmant la présence d'aberrations introduites (14 valeurs négatives signalées). Le rapport signale une forte corrélation avec `Annees_experience` (coefficient de **0.682**) et `Categorie_age` (coefficient de **0.815**). Aucune valeur manquante.
    *   `Annees_experience` : Comporte **40 valeurs manquantes** (0.1%). Un nombre notable de **5251 observations (13.1%** de la colonne) présentent une valeur de zéro (alert: `Annees_experience has 5251 (13.1%) zeros`), indiquant des individus sans expérience professionnelle. Elle est également fortement corrélée à `Age`.

*   **Variables Explicatives Catégorielles et Textuelles :**
    *   `Categorie_age` : Comporte **40 valeurs manquantes** (0.1%), probablement dues aux valeurs aberrantes de `Age`. Elle est logiquement très corrélée à `Age`. La catégorie la plus fréquente est "Adulte" (17615 occurrences).
    *   `Sexe`, `Milieu`, `Region_geographique`, `Niveau_education`, `CSP` : Ne présentent pas de valeurs manquantes. Leurs répartitions (ex: "Homme" 20817 pour `Sexe`, "Urbain" 23937 pour `Milieu`, "Secondaire" 13974 pour `Niveau_education`, "Ouvriers" 14975 pour `CSP`) sont conformes à la logique de génération. `Niveau_education` et `CSP` sont signalées comme fortement corrélées.
    *   `Etat_matrimonial` : Présente **35 valeurs manquantes** (0.1%). "Marié" est la catégorie la plus fréquente (21297).
    *   `Secteur_emploi` : A le plus grand nombre de valeurs manquantes avec **2610 occurrences (6.5%** des données pour cette colonne, alert: `Secteur_emploi has 2610 (6.5%) missing values`), ce qui est cohérent avec les "Inactifs" qui n'ont pas de secteur d'emploi. "Privé" est la catégorie la plus fréquente (17624).
    *   `Propriete_immobiliere` (**42 NaN**, 0.1%), `Vehicule_motorise` (**35 NaN**, 0.1%), `Terrain_agricole` (**31 NaN**, 0.1%), `Revenu_secondaire` (**38 NaN**, 0.1%) : Toutes ces variables binaires présentent un faible pourcentage de valeurs manquantes.
    *   `Terrain_agricole` est signalé par `ydata-profiling` comme étant déséquilibré (alert: `Terrain_agricole is highly imbalanced (56.0%)` - *Note: la valeur de 56.0% pour l'alerte d'imbalance est une interprétation de ydata-profiling; les données brutes indiquent environ 90.9% de 'Non'. L'alerte d'imbalance reste pertinente.*). Il est aussi fortement corrélé avec `CSP`.

*   **Colonnes Identifiants (Type Texte) :**
    *   `Adresse_Email` (**39956 valeurs distinctes** sur 40000) et `CIN` (**39999 valeurs distinctes** sur 40000) : Possèdent une très haute cardinalité (presque uniques pour chaque enregistrement), confirmant leur nature d'identifiants.

*   **Colonnes à considérer pour suppression (Redondance/Non Pertinence) :**
    *   `Revenu_Mensuel` : Identifié comme parfaitement corrélé avec `Revenu_Annuel` (coefficient de **1.000**). Sa suppression sera effectuée.
    *   `Adresse_Email` et `CIN` : Seront supprimées en raison de leur haute cardinalité et non-pertinence pour la prédiction.
    *   `Categorie_age` : Étant directement dérivée de `Age` et fortement corrélée (**0.815**), sa pertinence sera réévaluée. Si `Age` est conservé, `Categorie_age` pourrait être redondante.

*   **Corrélations Notables (basées sur la table de corrélation de Pearson du rapport) :**
    *   Variables les plus positivement corrélées avec `Revenu_Annuel` :
        *   `CSP` (coeff. **0.390**)
        *   `Niveau_education` (coeff. **0.305**)
        *   `Propriete_immobiliere` (coeff. **0.263**)
        *   `Vehicule_motorise` (coeff. **0.248**)
        *   `Annees_experience` (coeff. **0.196**)
        *   `Age` (coeff. **0.178**)
    *   Le rapport HTML complet de `ydata-profiling` devrait être consulté pour des métriques de corrélation plus adaptées aux variables mixtes (ex: Phik, Cramér's V).
    *   **Multicollinéarité** significative entre les variables explicatives (confirmée par les alertes de `ydata-profiling`) :
        *   `Age` et `Annees_experience` (**0.682**)
        *   `CSP` et `Niveau_education` (**0.640**)
        *   `CSP` et `Terrain_agricole` (**0.714**)
        *   `CSP` et `Secteur_emploi` (**0.473**)
        *   `CSP` et `Propriete_immobiliere` (**0.454**)
        *   `CSP` et `Vehicule_motorise` (**0.424**)
        *   `Niveau_education` et `Secteur_emploi` (**0.396**)
        *   `Niveau_education` et `Vehicule_motorise` (**0.335**)
        La gestion de cette multicollinéarité sera importante.

*   **Alertes du rapport `ydata-profiling` (Synthèse des 11 alertes mentionnées) :**
    1.  `Age` fortement corrélé avec `Annees_experience` et `Categorie_age`.
    2.  `Annees_experience` fortement corrélé avec `Age`.
    3.  `CSP` fortement corrélé avec `Niveau_education` et `Terrain_agricole`.
    4.  `Categorie_age` fortement corrélé avec `Age`.
    5.  `Niveau_education` fortement corrélé avec `CSP`.
    6.  `Revenu_Annuel` fortement corrélé avec `Revenu_Mensuel`.
    7.  `Revenu_Mensuel` fortement corrélé avec `Revenu_Annuel`.
    8.  `Terrain_agricole` fortement corrélé avec `CSP`.
    9.  `Terrain_agricole` est déséquilibré.
    10. `Secteur_emploi` a 6.5% de valeurs manquantes.
    11. `Annees_experience` a 13.1% de zéros.

**Conclusion de l'EDA et Prochaines Étapes :**
L'exploration des données nous a fourni une compréhension approfondie du dataset, de ses caractéristiques individuelles, de la qualité des données (présence de NaN, d'outliers intentionnels, et de zéros) et des relations et corrélations potentielles entre les variables. Les données générées semblent bien refléter les intentions initiales, y compris les imperfections et les relations attendues.
Les prochaines étapes se concentreront sur la préparation de ces données pour la modélisation :
1.  **Séparation des données** en ensembles d'apprentissage et de test.
2.  **Nettoyage des données**, ce qui inclura le traitement des valeurs manquantes et des valeurs aberrantes identifiées. (Les lignes dupliquées ne sont pas un problème ici).
3.  **Transformation des données**, comprenant la suppression des colonnes jugées non pertinentes ou redondantes, l'encodage approprié des variables catégorielles en format numérique, et potentiellement la normalisation ou la standardisation des variables numériques. La gestion de la multicollinéarité sera également considérée.
Ces opérations seront, autant que possible, intégrées dans des pipelines `scikit-learn` pour assurer un processus de prétraitement structuré, reproductible et applicable de manière cohérente aux différents sous-ensembles de données.

# 2. Préparation des Données

Après avoir exploré et compris notre dataset, cette section est dédiée à sa préparation en vue de la modélisation. Cela implique plusieurs étapes cruciales :
*   La suppression des colonnes jugées inutiles ou redondantes.
*   La séparation des données en ensembles d'apprentissage (`X_train`, `y_train`) et de test (`X_test`, `y_test`).
*   Le nettoyage des données, notamment le traitement des valeurs manquantes et des valeurs aberrantes.
*   La transformation des données, incluant l'encodage des variables catégorielles en format numérique et la normalisation/standardisation des variables numériques.

Conformément au cahier des charges, ces étapes de nettoyage et de transformation seront, autant que possible, intégrées dans des **pipelines `scikit-learn`** pour un traitement structuré et reproductible.

## 2.1 Suppression des Colonnes Inutiles et Définition des Features (X) et de la Cible (y)

Basé sur notre analyse exploratoire (section 1.6), nous allons d'abord supprimer les colonnes identifiées comme non pertinentes (identifiants uniques) ou redondantes. Ensuite, nous séparerons notre DataFrame en un ensemble de variables explicatives (features `X`) et la variable cible (`y`).

In [62]:

if 'df' in locals() and df is not None:
    df_processed = df.copy()

    TARGET_COL = 'Revenu_Annuel'
    y = df_processed[TARGET_COL]


    cols_to_drop_initial = ['Revenu_Mensuel', 'Adresse_Email', 'CIN', 'Categorie_age']
    
    actual_cols_to_drop = [col for col in cols_to_drop_initial if col in df_processed.columns]

    X = df_processed.drop(columns=[TARGET_COL] + actual_cols_to_drop, errors='ignore')

    print("Colonnes initialement supprimées de X :", actual_cols_to_drop)
    print(f"Nombre de colonnes restantes dans X avant séparation train/test : {X.shape[1]}")
    print("Liste des colonnes dans X :")
    print(X.columns.tolist())

    print(f"\nDimensions de X : {X.shape}")
    print(f"Dimensions de y : {y.shape}")

    print("\nAperçu de X (5 premières lignes) :")
    display(X.head())
    print("\nAperçu de y (5 premières valeurs) :")
    display(y.head())

else:
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé ou n'est pas défini.")

Colonnes initialement supprimées de X : ['Revenu_Mensuel', 'Adresse_Email', 'CIN', 'Categorie_age']
Nombre de colonnes restantes dans X avant séparation train/test : 13
Liste des colonnes dans X :
['Age', 'Sexe', 'Milieu', 'Region_geographique', 'Etat_matrimonial', 'Niveau_education', 'Annees_experience', 'CSP', 'Secteur_emploi', 'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole', 'Revenu_secondaire']

Dimensions de X : (40000, 13)
Dimensions de y : (40000,)

Aperçu de X (5 premières lignes) :


,Age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire
0,56,Homme,Rural,Centre,Marié,Fondamental,5.0,Agriculteurs,Privé,Non,Non,Oui,Non
1,46,Femme,Urbain,Centre,Célibataire,Supérieur,14.0,Cadres supérieurs,Public,Oui,Oui,Non,Non
2,32,Homme,Rural,Sud,Marié,Fondamental,8.0,Ouvriers,Informel,Non,Oui,Non,Non
3,60,Homme,Rural,Sud,Marié,Supérieur,17.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui
4,25,Femme,Urbain,Centre,Marié,Secondaire,3.0,Employés,Privé,Non,Non,Non,Non



Aperçu de y (5 premières valeurs) :


0       300.000000
1     78506.710725
2       395.248250
3    105781.533607
4     22016.268128
Name: Revenu_Annuel, dtype: float64

## 2.2 Séparation des Données en Ensembles d'Apprentissage et de Test

Avant de procéder au nettoyage et à la transformation plus poussée des données, il est crucial de diviser notre dataset (`X` et `y`) en deux sous-ensembles distincts :
*   Un **ensemble d'apprentissage** (`X_train`, `y_train`) : Ce sous-ensemble, représentant 70% des données, sera utilisé pour entraîner nos modèles de Machine Learning. C'est sur ces données que les modèles "apprendront" les relations entre les caractéristiques et la variable cible.
*   Un **ensemble de test** (`X_test`, `y_test`) : Ce sous-ensemble, représentant les 30% restants, sera mis de côté et utilisé uniquement à la fin du processus pour évaluer la performance des modèles entraînés sur des données qu'ils n'ont jamais vues auparavant. Cela permet d'obtenir une estimation impartiale de leur capacité à généraliser.

Cette séparation est fondamentale pour éviter le surapprentissage (overfitting), où un modèle apprendrait trop spécifiquement les données d'entraînement (y compris leur bruit) et performerait mal sur de nouvelles données. Nous utilisons la fonction `train_test_split` de `scikit-learn` pour cette opération, en spécifiant un `random_state` pour garantir la reproductibilité de la division.

In [70]:

from sklearn.model_selection import train_test_split

if 'X' in locals() and 'y' in locals(): 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.30, 
                                                        random_state=RANDOM_SEED)

    print("Dimensions des ensembles de données après séparation :")
    print(f"  X_train: {X_train.shape}")
    print(f"  y_train: {y_train.shape}")
    print(f"  X_test: {X_test.shape}")
    print(f"  y_test: {y_test.shape}")

    print("\nProportion des données dans chaque ensemble :")
    print(f"  Apprentissage (train) : {len(X_train) / len(X):.0%}")
    print(f"  Test : {len(X_test) / len(X):.0%}")
    
    print("\nAperçu de X_train (5 premières lignes) :")
    display(X_train.head())

else:
    print("ATTENTION : Les variables X et/ou y ne sont pas définies. Veuillez exécuter la cellule précédente (2.1).")

Dimensions des ensembles de données après séparation :
  X_train: (28000, 13)
  y_train: (28000,)
  X_test: (12000, 13)
  y_test: (12000,)

Proportion des données dans chaque ensemble :
  Apprentissage (train) : 70%
  Test : 30%

Aperçu de X_train (5 premières lignes) :


,Age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire
38015,34,Femme,Urbain,Sud,Célibataire,Secondaire,2.0,Employés,Privé,Oui,Non,Non,Non
2281,21,Homme,Rural,Est,Célibataire,Fondamental,2.0,Agriculteurs,Informel,Oui,Oui,Oui,Non
36629,56,Homme,Rural,Ouest,Célibataire,Fondamental,19.0,Ouvriers,Informel,Non,Non,Non,Non
6087,25,Femme,Urbain,Nord,Divorcé,Secondaire,2.0,Ouvriers,Privé,Non,Non,Non,Non
11792,35,Homme,Urbain,Ouest,Marié,Secondaire,7.0,Employés,Privé,Non,Non,Non,Non


## 2.3 Identification des Types de Colonnes pour les Pipelines de Prétraitement

Avant de construire nos pipelines, nous devons identifier clairement quelles colonnes de notre ensemble d'apprentissage `X_train` sont numériques et lesquelles sont catégorielles. Ces deux types de colonnes nécessiteront des traitements différents (par exemple, imputation et normalisation pour les numériques, imputation et encodage pour les catégorielles).

Nous identifierons également la colonne `Age` séparément car elle contient des valeurs aberrantes spécifiques (-5, 150) que nous pourrions vouloir traiter d'une manière particulière. De même pour `Annees_experience` qui contient des zéros et des valeurs manquantes.

In [66]:
if 'X_train' in locals():

    numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

    print("Colonnes Numériques Identifiées :")
    print(numerical_cols)
    print(f"Nombre de colonnes numériques : {len(numerical_cols)}")

    print("\nColonnes Catégorielles Identifiées :")
    print(categorical_cols)
    print(f"Nombre de colonnes catégorielles : {len(categorical_cols)}")

    if len(numerical_cols) + len(categorical_cols) == X_train.shape[1]:
        print("\nLa somme des colonnes numériques et catégorielles correspond bien au nombre total de colonnes dans X_train.")
    else:
        print("\nATTENTION : Incohérence dans le nombre de colonnes. Vérifiez les types.")
        print(f"Total colonnes X_train: {X_train.shape[1]}")
        print(f"Numériques + Catégorielles: {len(numerical_cols) + len(categorical_cols)}")

else:
    print("ATTENTION : X_train n'est pas défini. Veuillez exécuter les cellules précédentes.")

Colonnes Numériques Identifiées :
['Age', 'Annees_experience']
Nombre de colonnes numériques : 2

Colonnes Catégorielles Identifiées :
['Sexe', 'Milieu', 'Region_geographique', 'Etat_matrimonial', 'Niveau_education', 'CSP', 'Secteur_emploi', 'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole', 'Revenu_secondaire']
Nombre de colonnes catégorielles : 11

La somme des colonnes numériques et catégorielles correspond bien au nombre total de colonnes dans X_train.


## 2.4 Création des Pipelines de Prétraitement

Nous allons maintenant définir des pipelines distincts pour les variables numériques et catégorielles en utilisant `make_pipeline` et `ColumnTransformer` de Scikit-learn.

**Pour les variables numériques, le pipeline inclura :**
1.  **Traitement des valeurs manquantes (`SimpleImputer`) :** Nous utiliserons la médiane pour imputer les NaN, car elle est moins sensible aux outliers que la moyenne.
2.  **Traitement des valeurs aberrantes :** Pour la colonne `Age`, nous appliquerons une technique de "clipping" (plafonnement) pour ramener les valeurs aberrantes dans une plage plus raisonnable (par exemple, 18 à 70 ans). Pour les autres colonnes numériques, nous pourrions envisager `RobustScaler` qui est moins sensible aux outliers, ou d'autres techniques si nécessaire.
3.  **Normalisation/Standardisation (`StandardScaler` ou `RobustScaler`) :** Pour mettre toutes les variables numériques sur une échelle comparable, ce qui est important pour de nombreux algorithmes (Régression Linéaire, MLP, etc.). `StandardScaler` centre les données autour de 0 avec un écart-type de 1. `RobustScaler` utilise les quantiles et est plus robuste aux outliers.

**Pour les variables catégorielles, le pipeline inclura :**
1.  **Traitement des valeurs manquantes (`SimpleImputer`) :** Nous remplacerons les NaN par la valeur la plus fréquente de la colonne ou par une constante comme "Manquant".
2.  **Encodage (`OneHotEncoder`) :** Nous transformerons les catégories en un ensemble de colonnes binaires. Nous utiliserons `handle_unknown='ignore'` pour gérer les catégories rares qui pourraient apparaître dans l'ensemble de test mais pas dans l'ensemble d'apprentissage.

Ces pipelines individuels seront ensuite combinés à l'aide d'un `ColumnTransformer`.

In [71]:

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin 
import pandas as pd
import numpy as np


class AgeClipper(BaseEstimator, TransformerMixin):
    def __init__(self, min_age=18, max_age=75):
        self.min_age = min_age
        self.max_age = max_age
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed_np = np.copy(X) 
        X_transformed_np = np.clip(X_transformed_np, self.min_age, self.max_age)
        return X_transformed_np

if ('X_train' in locals() and 
    'numerical_cols' in locals() and
    'categorical_cols' in locals()): 

    age_pipeline = Pipeline([
        ('imputer_age', SimpleImputer(strategy='median')),
        ('scaler_age', StandardScaler())                       
    ])

    experience_pipeline = Pipeline([
        ('imputer_exp', SimpleImputer(strategy='median')),
        ('scaler_exp', StandardScaler()) 
    ])

    categorical_pipeline = Pipeline([
        ('imputer_cat', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop=None))
    ])

    age_col_for_transformer = [col for col in ['Age'] if col in X_train.columns]
    experience_col_for_transformer = [col for col in ['Annees_experience'] if col in X_train.columns]

    transformers_for_columntransformer = []
    if age_col_for_transformer:
        transformers_for_columntransformer.append(
            ('age_processing', age_pipeline, age_col_for_transformer)
        )
    if experience_col_for_transformer:
        transformers_for_columntransformer.append(
            ('experience_processing', experience_pipeline, experience_col_for_transformer)
        )
    if categorical_cols: 
        transformers_for_columntransformer.append(
            ('categorical_processing', categorical_pipeline, categorical_cols)
        )
    
    if transformers_for_columntransformer:
        preprocessor = ColumnTransformer(
            transformers=transformers_for_columntransformer,
            remainder='drop'
        )
        print("Preprocessor ColumnTransformer créé avec succès (version Étape 3).")
        from sklearn import set_config
        set_config(display='diagram') 
        display(preprocessor)
        set_config(display='text')
    else:
        print("ATTENTION : Aucun transformateur n'a été ajouté au ColumnTransformer.")
        preprocessor = None

else:
    print("ATTENTION : X_train ou les listes de colonnes numerical_cols/categorical_cols ne sont pas définis.")

Preprocessor ColumnTransformer créé avec succès (version Étape 3).


ColumnTransformer(transformers=[('age_processing',
                                 Pipeline(steps=[('imputer_age',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler_age',
                                                  StandardScaler())]),
                                 ['Age']),
                                ('experience_processing',
                                 Pipeline(steps=[('imputer_exp',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler_exp',
                                                  StandardScaler())]),
                                 ['Annees_experience']),
                                ('categorical_processing',
                                 Pipeline(steps=[('imputer_cat',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Sexe', 'Milieu', 'Region_geographique',
                                  'Etat_matrimonial', 'Niveau_education', 'CSP',
                                  'Secteur_emploi', 'Propriete_immobiliere',
                                  'Vehicule_motorise', 'Terrain_agricole',
                                  'Revenu_secondaire'])])

## 2.5 Application du Preprocessor et Récupération des N`sparse_output=False` dans `OneHotEncoder` :** C'est bien pour l'inspection.oms de Features

Le `preprocessor` étant défini, nous allons maintenant l'ajuster (`fit`) sur l'ensemble Si vous rencontrez des problèmes de mémoire plus tard avec un grand nombre de catégories après OHE, vous pourriez envisager de le d'entraînement `X_train` et l'utiliser pour transformer (`transform`) à la fois `X_train` et `X_test`.

Il est important de récupérer les noms des nouvelles features générées, notamment par le `OneHotEncoder passer à `True` et de travailler avec des matrices sparse (la plupart des modèles `sklearn` les acceptent). Pour `, pour pouvoir créer des DataFrames à partir des arrays NumPy retournés et inspecter les données transformées.

In [ ]:

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class AgeClipper(BaseEstimator, TransformerMixin):
    def __init__(self, min_age=18, max_age=75):
        self.min_age = min_age
        self.max_age = max_age
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed_np = np.copy(X)
        X_transformed_np = np.clip(X_transformed_np, self.min_age, self.max_age)
        return X_transformed_np

if ('preprocessor' in locals() and preprocessor is not None and 
    'X_train' in locals() and 'X_test' in locals() and
    'categorical_cols' in locals() and 
    'final_age_col' in locals() and 
    'final_experience_col' in locals()): 

    print("Application du preprocessor sur X_train (fit_transform)...")
    X_train_processed_np = preprocessor.fit_transform(X_train)
    
    print("Application du preprocessor sur X_test (transform)...")
    X_test_processed_np = preprocessor.transform(X_test)

    numeric_feature_names_processed = []
    if 'age_processing' in preprocessor.named_transformers_:
        numeric_feature_names_processed.extend([f"{col}_Processed" for col in final_age_col])

    if 'experience_processing' in preprocessor.named_transformers_:
        numeric_feature_names_processed.extend([f"{col}_Processed" for col in final_experience_col])
        
    try:
        onehot_transformer = preprocessor.named_transformers_['categorical_processing']
        onehot_encoder_step = onehot_transformer.named_steps['onehot']
        onehot_categorical_feature_names = list(onehot_encoder_step.get_feature_names_out(categorical_cols))
    except KeyError as e:
        print(f"Erreur de clé lors de la récupération des noms de features du OneHotEncoder: {e}")
        print("Vérifiez les noms des étapes ('categorical_processing', 'onehot') dans votre ColumnTransformer et le pipeline associé.")
        onehot_categorical_feature_names = [] 
    except Exception as e:
        print(f"Erreur générale lors de la récupération des noms de features du OneHotEncoder : {e}")
        onehot_categorical_feature_names = []


    final_feature_names = numeric_feature_names_processed + onehot_categorical_feature_names

    X_train_processed_df = pd.DataFrame(X_train_processed_np, 
                                        columns=final_feature_names, 
                                        index=X_train.index)
    
    X_test_processed_df = pd.DataFrame(X_test_processed_np, 
                                       columns=final_feature_names, 
                                       index=X_test.index)

    print(f"\nDimensions de X_train_processed_df : {X_train_processed_df.shape}")
    print("Aperçu de X_train_processed_df (5 premières lignes) :")
    display(X_train_processed_df.head())

    print(f"\nDimensions de X_test_processed_df : {X_test_processed_df.shape}")


    print(f"\nNombre de NaN dans X_train_processed_df : {X_train_processed_df.isnull().sum().sum()}")
    print(f"Nombre de NaN dans X_test_processed_df : {X_test_processed_df.isnull().sum().sum()}")

else:
    print("ATTENTION : Des variables nécessaires (preprocessor, X_train, X_test, categorical_cols, final_age_col, final_experience_col) ne sont pas définies. Veuillez exécuter les cellules précédentes.")

Application du preprocessor sur X_train (fit_transform)...
Application du preprocessor sur X_test (transform)...

Dimensions de X_train_processed_df : (28000, 36)
Aperçu de X_train_processed_df (5 premières lignes) :


,Age_Processed,Annees_experience_Processed,Sexe_Femme,Sexe_Homme,Milieu_Rural,Milieu_Urbain,Region_geographique_Centre,Region_geographique_Est,Region_geographique_Nord,Region_geographique_Ouest,Region_geographique_Sud,Etat_matrimonial_Célibataire,Etat_matrimonial_Divorcé,Etat_matrimonial_Marié,Etat_matrimonial_Veuf,Niveau_education_Fondamental,Niveau_education_Sans niveau,Niveau_education_Secondaire,Niveau_education_Supérieur,CSP_Agriculteurs,CSP_Cadres supérieurs,CSP_Employés,CSP_Inactifs,CSP_Ouvriers,CSP_Professions intermédiaires,Secteur_emploi_Informel,Secteur_emploi_Privé,Secteur_emploi_Public,Propriete_immobiliere_Non,Propriete_immobiliere_Oui,Vehicule_motorise_Non,Vehicule_motorise_Oui,Terrain_agricole_Non,Terrain_agricole_Oui,Revenu_secondaire_Non,Revenu_secondaire_Oui
38015,-0.481264,-0.847681,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2281,-1.445720,-0.847681,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
36629,1.150893,0.879334,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6087,-1.148964,-0.847681,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
11792,-0.407075,-0.339736,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0



Dimensions de X_test_processed_df : (12000, 36)

Nombre de NaN dans X_train_processed_df : 0
Nombre de NaN dans X_test_processed_df : 0


# 3. Création et Validation des Modèles

Après avoir préparé nos données, nous passons maintenant à l'étape de modélisation. L'objectif est de :
1.  Sélectionner plusieurs algorithmes de régression.
2.  Ajuster (fine-tune) leurs hyperparamètres en utilisant la technique de validation croisée sur l'ensemble d'entraînement (`X_train_processed_df`, `y_train`).
3.  Comparer les performances des différents modèles sur la base des métriques MAE, RMSE, et R².
4.  Choisir le modèle le plus performant.

Les modèles à considérer sont : Régression Linéaire, Arbres de Décision, Forêts d'Arbres Décisionnels, Gradient Boosting, et Réseaux de Neurones Multi-couches (MLPRegressor).

## 3.1 Modèle 1 : Régression Linéaire (`LinearRegression`)

Nous commençons notre exploration des modèles de régression avec un algorithme fondamental : la Régression Linéaire. Ce modèle cherche à établir une relation linéaire entre les variables explicatives (features) et la variable cible (revenu annuel).

Bien que la Régression Linéaire soit simple et interprétable, ses performances peuvent être limitées si les relations sous-jacentes dans les données ne sont pas linéaires ou si les hypothèses du modèle (comme l'absence de forte multicollinéarité ou l'homoscédasticité des erreurs) ne sont pas respectées.

Pour ce modèle, le cahier des charges indique qu'il n'y a **aucun hyperparamètre à ajuster** via `GridSearchCV`. Nous allons donc l'entraîner directement sur les données d'apprentissage et évaluer ses performances initiales en utilisant la validation croisée pour obtenir une estimation plus robuste des métriques MAE, RMSE et R².

In [76]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
import time 

if ('X_train_processed_df' in locals() and 'y_train' in locals() and
    'X_test_processed_df' in locals() and 'y_test' in locals()):

    print("--- Modèle : Régression Linéaire (Évaluation sur X_test) ---")

    # 1. Initialisation du modèle
    linear_model_test_split = LinearRegression()

    # 2. Entraînement du modèle sur l'ensemble d'entraînement complet prétraité
    print("\nEntraînement du modèle sur X_train_processed_df...")
    start_time = time.time()
    linear_model_test_split.fit(X_train_processed_df, y_train)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Entraînement terminé en {training_time:.4f} secondes.")

    # 3. Prédictions sur l'ensemble de test prétraité
    print("\nPrédictions sur X_test_processed_df...")
    predictions_test = linear_model_test_split.predict(X_test_processed_df)

    # 4. Évaluation des performances sur l'ensemble de test
    mae_test = mean_absolute_error(y_test, predictions_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, predictions_test))
    r2_test = r2_score(y_test, predictions_test)

    print("\n--- Évaluation du Modèle sur l'Ensemble de Test (X_test) ---")
    print(f"  MAE (Mean Absolute Error)        : {mae_test:.2f} DH")
    print(f"  RMSE (Root Mean Squared Error)   : {rmse_test:.2f} DH")
    print(f"  R² (Coefficient de détermination): {r2_test:.4f}")

    # Afficher un aperçu des prédictions vs valeurs réelles pour vérification
    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (sur X_test) ---")
    df_predictions = pd.DataFrame({'Valeur_Réelle': y_test.values, 
                                   'Valeur_Prédite': predictions_test})
    display(df_predictions.head(10)) 
else:
    print("ATTENTION : Les données prétraitées (X_train_processed_df, y_train, X_test_processed_df, y_test) ne sont pas définies.")
    print("Veuillez exécuter les cellules de prétraitement des données (jusqu'à 2.5 inclus avec les modifications appropriées).")

--- Modèle : Régression Linéaire (Évaluation sur X_test) ---

Entraînement du modèle sur X_train_processed_df...
Entraînement terminé en 0.0612 secondes.

Prédictions sur X_test_processed_df...

--- Évaluation du Modèle sur l'Ensemble de Test (X_test) ---
  MAE (Mean Absolute Error)        : 8112.47 DH
  RMSE (Root Mean Squared Error)   : 17097.94 DH
  R² (Coefficient de détermination): 0.7394

--- Aperçu des Prédictions vs Valeurs Réelles (sur X_test) ---


,Valeur_Réelle,Valeur_Prédite
0,300.000000,-4420.060133
1,9514.349720,9532.598402
2,13886.129297,30755.694900
3,12291.185637,12126.760540
4,10524.116430,13347.567358
5,20478.219738,20639.733890
6,6465.972785,22327.635116
7,29689.598439,26558.987518
8,9097.518019,18506.481355
9,107554.335002,111933.826843
